In [1]:
import w2vAida
import analogy_completion
import pandas as pd
import numpy as np

In [2]:
## CONFIG
model_version = "w2v"   # one of s2v, aida, or w2v
run_completion_experiment = False
run_ranking_experiment = True
exclude_minority_vectores = False

In [3]:
## logging
log_path = "/opt3/home/lofi/github/SimilarityAndAnalogy/analogy/btw17/log/"
log_filename = model_version+"_"+("NoMinority" if exclude_minority_vectores else "Minority")+".log"

import logging
from logging.handlers import RotatingFileHandler
import sys

log = logging.getLogger('')
log.setLevel(logging.DEBUG)
format = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")

#ch = logging.StreamHandler(sys.stdout)
#ch.setFormatter(format)
#log.addHandler(ch)

#fh = logging.handlers.RotatingFileHandler(log_path+log_filename, maxBytes=(1048576*5), backupCount=7)
fh = logging.FileHandler(log_path+log_filename, mode="w")
fh.setFormatter(format)

log.addHandler(fh)

In [4]:
### VERSION on IS69
if (model_version == "s2v"):
    model_file = "/opt3/home/pratima/thesis_final/models/s2v.model.bin"
    vocab_file = "/opt3/home/lofi/word2vec_models/s2v.vocab"
    delimeter = "|"
elif (model_version == "aida"):
    model_file = "/opt3/home/pratima/thesis_final/models/aida.model.bin"
    vocab_file = "/opt3/home/lofi/word2vec_models/aida.vocab"
    delimeter = ":"
logging.debug("version: "+model_version)
golddata_file = "/opt3/home/lofi/github/SimilarityAndAnalogy/analogy/testData/AGS/AGS-V02.txt"
output_file = "/opt3/home/lofi/github/SimilarityAndAnalogy/analogy/testData/AGS/results//test.txt"

In [ ]:
### LOCAL VERSION

#model_file = "D:/data/analogy/pratima_w2v_models/aida.model.bin"
#model_file = "D:/data/analogy/pratima_w2v_models/s2v.model.bin"
#vocab_file = "D:/data/analogy/pratima_w2v_models/aida.vocab"
#golddata_file = "../testData/AGS/AGS-V02.txt"
#output_file = "./result/test.txt"

In [5]:
### LOAD AGS

def loadGoldData(dataset):
    with open(dataset, 'r') as file:
        lines = file.readlines()
        simGold = []
        i = 0
        while i < len(lines):
            line = lines[i].strip()
            i += 1
            if line.startswith(":"):
                continue
            if line.startswith("\n"):
                i += 1
                continue
            splits = line.split(" ")
            correctedsplits = [ ]
            correctedsplits.append(splits[0])
            correctedsplits.append(splits[1])
            correctedsplits.append(splits[2])
            correctedsplits.append(splits[3])
            correctedsplits.append(splits[4])
            correctedsplits.append(splits[5])
            correctedsplits.append(splits[6])
            correctedsplits.append(splits[7])
            #correctedsplits.append(splits[8])
            simGold.append(correctedsplits)
            #print(correctedsplits)
        return simGold
    
analogydataset = loadGoldData(golddata_file)

In [6]:
### Load Training Model. Note: s2v needs | delimeter, 
model=w2vAida.Word2Vec.load_word2vec_format(fname=model_file, binary=True, fvocab=vocab_file, delimeter="|")

load local with delimeter |


In [ ]:
# just a test 
if (model_version=="s2v"):
    print(model.n_similarity(['sushi|noun', 'shop|noun'], ['japanese|noun', 'restaurant|noun']))

In [ ]:
### completion query!
if run_completion_experiment:
    for value in analogydataset:
        temp = model.most_similar_new(positive=[value[2], value[1]], negative=[value[0]],topn=1)
        for values in temp:
            if ':' in values[0]:
                v = values[0]
                v1 = v.split(':')
                similarityvalue=v1[0]
            else:
                similarityvalue=values[0]
        print("{}:{} :: {}:{}  >> {}".format(value[0], value[1], value[2], value[3], similarityvalue))

In [ ]:
r_columns=('a1','a2','b1','b2','ags_rating','model_rating')
result = pd.DataFrame(columns=r_columns)
result_file_name=log_path+model_version+"_"+("NoMinority" if exclude_minority_vectores else "Minority")+".result.csv"

In [ ]:
### RANKING EXPERIMENT
if run_ranking_experiment:

    
    duplicatecount=0
    count=0
    countlines=0
    previouscheckvalue=0
    total=0
    nextval = 0

    for value in analogydataset:
        #if(value[6]==word):
        r_similarity = model.n_similarity_new([value[0], value[1]],[value[2],value[3]])
        print("{}:{} :: {}:{}  >> {}    reference rating {}".format(value[0], value[1], 
                                                                    value[2], value[3], r_similarity, value[4]))

        countlines+=1
        originalRating=value[4]
        originalcheckvalue=value[0]
       
        row = pd.DataFrame([[value[0],value[1],value[2],value[3],value[4],r_similarity]],columns=r_columns)
        result=result.append(row)
        result.to_csv(result_file_name, encoding='utf-8')
              

tired:sleep :: thirsty:drink  >> 0.8244511215034911    reference rating 5
tired:sleep :: sad:cry  >> 0.7215932798480629    reference rating 4.7
tired:sleep :: hungry:eat  >> 0.767812951596686    reference rating 4.6
tired:sleep :: fresh:work  >> 0.6284949519846539    reference rating 3.6
tired:sleep :: lazy:couch  >> 0.7497785577716891    reference rating 3
tired:sleep :: bored:read  >> 0.7080494526116101    reference rating 2.6
tired:sleep :: happy:sit  >> 0.7837585912811622    reference rating 1.2
lifejacket:boat :: belt:car  >> 0.8479072867781147    reference rating 5
lifejacket:boat :: seatbelt:airplane  >> 0.6844971447127233    reference rating 5
lifejacket:boat :: airbag:car  >> 0.8388618580291823    reference rating 4.6


In [ ]:
print(model.vocab["apple|ent"].count)

In [ ]:
len(model.index2word)